In [ ]:
# IMPORT

import requests

import pandas as pd

import base64


## TEST API BRICKOGNIZE

doc JSON =  https://api.brickognize.com/openapi.json


In [ ]:

# Charger et transformation de l'image
image_path = "/Users/Cassille/code/Cassille/LegoLAS/notebooks/test.jpeg" # chemin du fichier
with open(image_path, "rb") as img_file: # ouvrir le fichier en binaire
    img_data = img_file.read() # lecture du fichier
    encoded_image = encoded_string = base64.b64encode(img_data).decode("utf-8") # pour mettre dans le dataframe

    # Générer un lien HTML
html_code = f'<img src="data:image/jpeg;base64,{encoded_image}" alt="LEGO Brick">'

# Définir l'URL de l'API
url = "https://api.brickognize.com/predict/"

# Préparer les fichiers pour la requête
files = {
    'query_image': ('image.jpg', img_data, 'image/jpeg') # contenu binaire du fichier; type image JPEG
}

# Définir les en-têtes
headers = {
    'accept': 'application/json'
}

# Envoyer la requête POST
response = requests.post(url, headers=headers, files=files)

# Vérifier le statut de la réponse
if response.status_code == 200:
    # Afficher la réponse JSON
    print(response.json())
else:
    print(f"Erreur {response.status_code}: {response.text}")


In [ ]:
#Ne pas
response= response.json().get('items')
response

In [ ]:
df =pd.json_normalize(response, sep='_')


#Extraire les URLs des sites externes car sinon URL non exploitable directement
# drop des colonnes external_sites car renommée , type et score

df['external_site_name'] = df['external_sites'].apply(lambda x: x[0]['name'] if x else None)
df['external_site_url'] = df['external_sites'].apply(lambda x: x[0]['url'] if x else None)
df=df.drop(columns = ['external_sites','type','score'])

# ajouter l'image analyser dans le DF
#df['mon_image']=html_code

df['mon_image']=html_code

# trier mon DF

nouvel_ordre=['mon_image','id',"name",'category','img_url','external_site_name','external_site_url']
df=df[nouvel_ordre]

df

In [ ]:
html_code

In [ ]:
# ici rajouter la possibilité de choisir le model de pièce de lego avec index ?

In [ ]:
# IMPORT

import requests
import pandas as pd
import base64

# CLASSIFICATION VIA API BRICKOGNIZE

def classification('image_path)'':
    """
    1- Création du chemin de l'image pour etre envoyé à l'API


    """

    ## Charger et transformation de l'image pour insertion en le dataframe
    #image_path = "/Users/Cassille/code/Cassille/LegoLAS/notebooks/test.jpeg" # chemin du fichier
    with open(image_path, "rb") as img_file: # ouvrir le fichier en binaire
        img_data = img_file.read() # lecture du fichier



    # si besoin pour mettre l'image dans le df
    #     encoded_image = base64.b64encode(img_data).decode("utf-8") # pour mettre dans le dataframe
    # # Générer un lien HTML pour notre image
    # html_code = f'<img src="data:image/jpeg;base64,{encoded_image}" alt="LEGO Brick">'


    ## Définir l'URL de l'API
    url = "https://api.brickognize.com/predict/"

    # Préparer les fichiers pour la requête
    files = {
        'query_image': ('image.jpg', img_data, 'image/jpeg') # contenu binaire du fichier; type image JPEG
    }

    # Définir les en-têtes
    headers = {
        'accept': 'application/json'
    }

    # Envoyer la requête POST
    response = requests.post(url, headers=headers, files=files)

    # Vérifier le statut de la réponse
    if response.status_code == 200:
        # Afficher la réponse JSON
        response_json=response.json()
        response_items=response_data['items']
        df = pd.DataFrame(response_items)

        return df

    else:
        print(f"Erreur {response.status_code}: {response.text}")


In [1]:
classification('/Users/Cassille/code/Cassille/LegoLAS/notebooks/images/image.jpeg')


NameError: name 'classification' is not defined

##  ❌TEST STOCKAGE DANS CSV - NON UTILE ❌

In [ ]:
import os
import pandas as pd
import requests
import base64

# Définir le dossier de stockage
dossier = "lego_classification"
os.makedirs(dossier, exist_ok=True)  # Crée le dossier s'il n'existe pas

# Vérifier si le fichier CSV existe, sinon le créer
csv_path = os.path.join(dossier, "tableau.csv")
if not os.path.exists(csv_path):
    df_vierge = pd.DataFrame(columns=["image_path", 'mon_image','id',"name",'category','img_url','external_site_name','external_site_url',"count"])
    df_vierge.to_csv(csv_path, index=False)
    print("✅ Fichier tableau.csv créé !")


In [ ]:

# Classification de l'image :
image_path = "/Users/Cassille/code/Cassille/LegoLAS/notebooks/images/image.jpeg"

# Charger et encoder l’image
with open(image_path, "rb") as img_file:
    img_data = img_file.read()
    encoded_image = base64.b64encode(img_data).decode("utf-8")

# Générer un lien HTML
html_code = f'<img src="data:image/jpeg;base64,{encoded_image}" alt="LEGO Brick">'

# Définir l'URL de l'API
url = "https://api.brickognize.com/predict/"
headers = {'accept': 'application/json'}
files = {'query_image': ('image.jpg', img_data, 'image/jpeg')}

# Envoyer la requête POST
response = requests.post(url, headers=headers, files=files)

if response.status_code == 200:
    response_data = response.json()  # Extraire le JSON
    items = response_data.get('items')

    # Ajouter la nouvelle donnée au DataFrame existant
    df = pd.read_csv(csv_path)
    df = pd.concat([df, pd.DataFrame(response_data['items'])], ignore_index=True)

    # Enregistrer les mises à jour dans le fichier CSV
    df.to_csv(csv_path, index=False)
    print(f"✅ Image {image_path} analysée et ajoutée au tableau !")

else:
    print(f"❌ Erreur {response.status_code}: {response.text}")


In [ ]:
items = response_data.get('items')
ids = [item["name"] for item in items]

print(ids)

In [ ]:
response_data = response.json()
pd.DataFrame(response_data['items'])

In [ ]:
df = pd.read_csv("lego_classification/tableau.csv")
df

In [ ]:
def classification(image_path):

    """
    Analyse une image LEGO via l'API Brickognize et retourne un DataFrame contenant les résultats.

    Args:
        image_path (str): Chemin d'accès à l'image à analyser.

    Returns:
        pd.DataFrame: Tableau contenant les informations des pièces LEGO détectées, incluant :
            - id (str) : Identifiant de la pièce LEGO
            - name (str) : Nom de la pièce LEGO
            - img_url (str) : URL de l'image de la pièce LEGO
            - category (str) : Catégorie de la pièce LEGO
            - external_site_name (str) : Nom du site externe lié à la pièce
            - external_site_url (str) : URL du site externe

    Raises:
        ValueError: Si la requête à l'API échoue ou si aucune pièce LEGO n'est détectée.

    Example:
        >>> df = classification("/path/to/image.jpeg")
        >>> print(df.head())
    """

    with open(image_path, "rb") as img_file:
        img_data = img_file.read()

    url = "https://api.brickognize.com/predict/"

    files = {
        'query_image': ('image.jpg', img_data, 'image/jpeg')
    }

    headers = {
        'accept': 'application/json'
    }

    response = requests.post(url, headers=headers, files=files)

    if response.status_code == 200:
        response_json=response.json()
        response_items=response_json['items']
        df = pd.DataFrame(response_items)

        return df

    else:
        print(f"Erreur {response.status_code}: {response.text}")

In [46]:
classification('/Users/Cassille/code/Cassille/LegoLAS/notebooks/images/image.jpeg')

,id,name,img_url,external_sites,category,type,score
0,3001,Brick 2 x 4,https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Brick,part,0.843082
1,3001special,"Brick 2 x 4 special (special bricks, test bric...",https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Brick,part,0.671341
2,3011,"Duplo, Brick 2 x 4",https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...","DUPLO, Brick",part,0.573433
3,29541,"Brick, Soft 2 x 4",https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Soft Bricks,part,0.566717
4,bhol04,Brick 2 x 4 without Bottom Tubes,https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Brick,part,0.548071
5,3001old,Brick 2 x 4 without Cross Supports,https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Brick,part,0.533141
6,bslot04,"Brick 2 x 4 without Bottom Tubes, Slotted (wit...",https://storage.googleapis.com/brickognize-sta...,"[{'name': 'bricklink', 'url': 'https://www.bri...",Classic,part,0.501408
